In [1]:
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras import Sequential
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from keras.optimizers import SGD
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D, LeakyReLU
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.utils import to_categorical
from keras.utils.generic_utils import get_custom_objects

In [2]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=.2)

In [3]:
y_train = to_categorical(y_train)
y_val = to_categorical(y_val)
y_test = to_categorical(y_test)

In [4]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape, x_test.shape, y_test.shape

((40000, 32, 32, 3),
 (40000, 10),
 (10000, 32, 32, 3),
 (10000, 10),
 (10000, 32, 32, 3),
 (10000, 10))

In [5]:
train_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
val_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)
test_generator = ImageDataGenerator(rotation_range=2, horizontal_flip=True)

train_generator.fit(x_train)
val_generator.fit(x_val)
test_generator.fit(x_test)

In [6]:
def alexnet(activation):
    AlexNet = Sequential()

    AlexNet.add(Conv2D(filters=96, input_shape=(32, 32, 3), kernel_size=(11, 11), strides=(4, 4), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=256, kernel_size=(5, 5), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=384, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))

    AlexNet.add(Conv2D(filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same'))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same'))

    AlexNet.add(Flatten())
    AlexNet.add(Dense(4096, input_shape=(32, 32, 3)))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(4096))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(1000))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation(activation))
    AlexNet.add(Dropout(0.5))

    AlexNet.add(Dense(10))
    AlexNet.add(BatchNormalization())
    AlexNet.add(Activation('softmax'))

    return AlexNet

In [7]:
def gelu(x):
    return 0.5 * x * (1 + tf.tanh(tf.sqrt(2 / np.pi) * (x + 0.044715 * tf.pow(x, 3))))
get_custom_objects().update({'gelu': Activation(gelu)})

def swish(x):
    return x * tf.sigmoid(x)
get_custom_objects().update({'swish': Activation(swish)})

get_custom_objects().update({'leaky-relu': Activation(LeakyReLU(alpha=0.2))})

# act_func = ['tanh', 'relu', 'leaky-relu', 'elu', 'selu', 'gelu', 'swish']

model = alexnet('tanh')

batch_size = 128
epochs = 50

model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator.flow(x_train, y_train, batch_size=batch_size), epochs=epochs,
                      validation_data=val_generator.flow(x_val, y_val, batch_size=batch_size), verbose=1)

Epoch 1/50
313/313 [==============================] - 55s 107ms/step - loss: 2.0113 - accuracy: 0.2781 - val_loss: 1.6773 - val_accuracy: 0.3938
Epoch 2/50
313/313 [==============================] - 31s 99ms/step - loss: 1.5707 - accuracy: 0.4441 - val_loss: 1.5732 - val_accuracy: 0.4456
Epoch 3/50
313/313 [==============================] - 32s 103ms/step - loss: 1.4513 - accuracy: 0.4934 - val_loss: 1.4960 - val_accuracy: 0.4781
Epoch 4/50
313/313 [==============================] - 32s 103ms/step - loss: 1.3405 - accuracy: 0.5370 - val_loss: 1.4514 - val_accuracy: 0.4941
Epoch 5/50
313/313 [==============================] - 33s 104ms/step - loss: 1.2771 - accuracy: 0.5613 - val_loss: 1.3915 - val_accuracy: 0.5160
Epoch 6/50
313/313 [==============================] - 32s 102ms/step - loss: 1.2134 - accuracy: 0.5865 - val_loss: 1.3260 - val_accuracy: 0.5472
Epoch 7/50
313/313 [==============================] - 32s 104ms/step - loss: 1.1536 - accuracy: 0.6046 - val_loss: 1.5813 - val_acc

In [8]:
print(history.history)
y_pred = np.argmax(model.predict(x_test), axis=1)
y_true = np.argmax(y_test,axis=1)

print(y_pred.shape)
print(y_true.shape)

print(np.sum(y_pred == y_true) / y_pred.shape[0])

{'loss': [1.8192692995071411, 1.5386582612991333, 1.4236712455749512, 1.3322482109069824, 1.2635610103607178, 1.2008086442947388, 1.1454763412475586, 1.1024616956710815, 1.054668664932251, 1.0149914026260376, 0.9743750095367432, 0.9400891661643982, 0.9081237316131592, 0.8748908042907715, 0.8462015390396118, 0.8153714537620544, 0.7850127220153809, 0.7526926398277283, 0.722160279750824, 0.6945837140083313, 0.6739773154258728, 0.6453880071640015, 0.6238464117050171, 0.5927892923355103, 0.5749105215072632, 0.5451666712760925, 0.5220342874526978, 0.501533567905426, 0.48165076971054077, 0.46648895740509033, 0.4373399317264557, 0.41826170682907104, 0.39944976568222046, 0.38074493408203125, 0.35684046149253845, 0.3436119854450226, 0.3287675678730011, 0.31865718960762024, 0.2972946763038635, 0.28698861598968506, 0.27228519320487976, 0.25497323274612427, 0.24100996553897858, 0.2348290979862213, 0.22054602205753326, 0.20942416787147522, 0.19202469289302826, 0.19090473651885986, 0.1787385940551757